<a href="https://colab.research.google.com/github/SteveGadd/animal-classifier-cnn/blob/main/animal_classifier_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Resize, Compose
from torchvision import models


import matplotlib.pyplot as plt
import os
import kagglehub

torch.manual_seed(1864)

# Download latest version
path = kagglehub.dataset_download("iamsouravbanerjee/animal-image-dataset-90-different-animals")
root_dir = os.path.expanduser("/kaggle/input/animal-image-dataset-90-different-animals/animals/animals")

transform = Compose([
    Resize((224, 224)),  # Resize to a fixed size
    ToTensor()
])

# Load dataset
dataset = ImageFolder(root=root_dir, transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, val_data = random_split(dataset, [train_size, val_size])
# Create DataLoader
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [2]:
print(f"Total samples: {len(dataset)}")

# Classes
print(f"Number of classes: {len(dataset.classes)}")

# Sample shape
img, label = dataset[0]
print(f"Image shape: {img.shape}")
print(f"Label index: {label}")
print(f"Label name: {dataset.classes[label]}")

Total samples: 5400
Number of classes: 90
Image shape: torch.Size([3, 224, 224])
Label index: 0
Label name: antelope


In [3]:
#ImageClassifier Neural Network
class ImageClassifier(nn.Module):
  def __init__(self):
      super().__init__()
      self.model = nn.Sequential(
      nn.Conv2d(3, 32, (3,3)),     # 222x222
      nn.ReLU(),
      nn.MaxPool2d(2),             # 111x111
      nn.Conv2d(32, 64, (3,3)),    # 109x109
      nn.ReLU(),
      nn.MaxPool2d(2),             # 54x54
      nn.Conv2d(64, 64, (3,3)),    # 52x52
      nn.ReLU(),
      nn.MaxPool2d(2),             # 26x26
      nn.Flatten(),
      nn.Linear(64*26*26, 90)
  )
  def forward(self,x):
    return self.model(x)


In [4]:
clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(),lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [5]:
# Training loop
epochs = 10

for epoch in range(epochs):
    for batch, (X, y) in enumerate(train_loader):
        # Move data to GPU
        X, y = X.to('cuda'), y.to('cuda')

        # Forward pass
        pred = clf(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        opt.zero_grad()  # Clear old gradients
        loss.backward()  # Calculate gradients
        opt.step()       # Update weights

        # Print progress
        if batch % 100 == 0:
            print(f"Epoch {epoch}, Batch {batch}, Loss: {loss.item():.4f}")

Epoch 0, Batch 0, Loss: 4.4956
Epoch 0, Batch 100, Loss: 4.4854
Epoch 1, Batch 0, Loss: 4.1881
Epoch 1, Batch 100, Loss: 4.3880
Epoch 2, Batch 0, Loss: 4.0688
Epoch 2, Batch 100, Loss: 4.0768
Epoch 3, Batch 0, Loss: 3.2647
Epoch 3, Batch 100, Loss: 3.7306
Epoch 4, Batch 0, Loss: 2.7674
Epoch 4, Batch 100, Loss: 2.2553
Epoch 5, Batch 0, Loss: 2.0205
Epoch 5, Batch 100, Loss: 1.0718
Epoch 6, Batch 0, Loss: 0.7111
Epoch 6, Batch 100, Loss: 0.8207
Epoch 7, Batch 0, Loss: 0.6395
Epoch 7, Batch 100, Loss: 0.5218
Epoch 8, Batch 0, Loss: 0.3857
Epoch 8, Batch 100, Loss: 0.4351
Epoch 9, Batch 0, Loss: 0.3677
Epoch 9, Batch 100, Loss: 0.6102


In [6]:
# Quick accuracy check
correct = 0
total = 0
clf.eval()
with torch.no_grad():
    for X, y in val_loader:
        X, y = X.to('cuda'), y.to('cuda')
        pred = clf(X)
        correct += (pred.argmax(1) == y).sum().item()
        total += y.size(0)

print(f"Accuracy: {100*correct/total:.2f}%")

Accuracy: 95.00%
